In [1]:
from tqdm import tqdm

import csv
from decimal import *
from glob import glob
import json

In [2]:
datadir = "data/"

In [3]:
# DANGER getcontext().prec = 2    # Two decimal digits -- pennies

In [4]:
prefixes = {
    "E": "content__IDV__",
    "F": "content__award__",
    "K": "content__award__",
    "N": "content__award__",
    "X": "content__award__",
}

# hahaha no

In [5]:
prefixes = {
    "IDV": "content__IDV__",
    "AWARD": "content__award__",
    "OTHERTRANSACTIONAWARD": "content__OtherTransactionAward__contractDetail__",
}

In [6]:
valueswanted = [
    "dollarValues__obligatedAmount",
    "dollarValues__baseAndAllOptionsValue",
    "totalDollarValues__totalObligatedAmount",
    "totalDollarValues__totalBaseAndAllOptionsValue",
]

In [7]:
rawfilenames = list(sorted(glob(datadir + "*.json")))
basefilenames = {}
for rawfilename in tqdm(rawfilenames):
    basefilename = rawfilename.replace("\\", "/").replace(datadir, "")
    localdict = {}
    localdict['filename'] = basefilename
    localdict['date'] = basefilename.split("contracts-")[-1].split("_")[0]
    reason = basefilename.split(".json")[0].split("_")[-1]
    localdict['reason'] = reason
    with open(rawfilename, "r", encoding="utf-8") as infile:
        rawjson = json.load(infile)
    localdict['entries'] = len(rawjson)
    for valuewanted in valueswanted:
        localdict[valuewanted] = Decimal(0)
    for entry in rawjson:
        prefix = prefixes[entry['contract_type']]
        for valuewanted in valueswanted:
            localdict[valuewanted] += Decimal(entry[prefix + valuewanted].replace(",", "").replace("$", ""))
    
    basefilenames[basefilename] = localdict

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:11<00:00, 26.10it/s]


In [8]:
with open("file-summaries.csv", "w", encoding="utf-8", newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(list(localdict.keys()))
    for basefilename in basefilenames:
        writer.writerow(list(basefilenames[basefilename].values()))

In [9]:
basefilename

'contracts-2025-03-20_X.json'

In [10]:
rawfilename

'data\\contracts-2025-03-20_X.json'

In [11]:
str(localdict['totalDollarValues__totalObligatedAmount'])

'3214536.38'